In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from PIL import Image, ImageDraw

In [2]:
def black_noise_wrapper(amount=0.05):
    def black_noise(image, rows, cols):
        num_noise = np.ceil(amount * rows * cols)

        # random pixel
        coords = [
            np.random.randint(0, i - 1, int(num_noise)) for i in [rows, cols]
        ]
        image[coords[0], coords[1], :] = 0

        return image

    return black_noise

def color_noise_wrapper(amount=0.05, rgb=[125, 200, 230]):
    def color_noise(image, rows, cols):
        num_noise = np.ceil(amount * rows * cols)

        # random pixel
        coords = [
            np.random.randint(0, i - 1, int(num_noise)) for i in [rows, cols]
        ]
        image[coords[0], coords[1], :] = rgb

        return image
    
    return color_noise

def polygon_noise_wrapper(num_polygons=10000, min_radius=5, max_radius=20):
    def polygon_noise(image, cols, rows):
        image_size = (cols, rows)
        noise_image = Image.new("RGBA", image_size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(noise_image)

        for _ in range(num_polygons):
            # 중심점 설정
            center_x = np.random.randint(0, image_size[0])
            center_y = np.random.randint(0, image_size[1])

            # 다각형 꼭짓점 생성
            polygon = []
            for _ in range(np.random.randint(3, 6)):
                angle = np.random.uniform(0, 2 * np.pi)
                radius = np.random.uniform(min_radius, max_radius)
                x = int(center_x + np.cos(angle) * radius)
                y = int(center_y + np.sin(angle) * radius)
                polygon.append((x, y))

            # 투명도 설정
            transparency = np.random.randint(30, 100)
            # 다각형 그리기
            draw.polygon(polygon, fill=(140, 215, 245, transparency))

        original_image = Image.fromarray(image).convert("RGBA")
        composite_image = Image.alpha_composite(original_image, noise_image)

        return np.array(composite_image)[:, :, :3]

    return polygon_noise


In [ ]:
# -- 수정할 부분 --

img_dir = "/data/ephemeral/home/data/medical/img/train"  # train set 폴더 경로
save_dir = "/data/ephemeral/home/data/medical/aug"  # 저장할 폴더

img_idx = 0  # 증강 적용할 이미지 인덱스 (0 ~ 99)

amount = 0.1  # 전체 픽셀에서 노이즈로 바꿀 비율 (0.0 ~ 1.0)
rgb = [0, 0, 0] # color noise r, g, b
num_polygons = 10000 # polygon num
min_radius, max_radius = 5, 20  # polygon size
factor = -0.3  # 이미지 밝기 조절 (음수: 어둡게, 양수: 밝게)

# --------------

tf_black_noise = A.Lambda(image=black_noise_wrapper(amount), p=1)
tf_color_noise = A.Lambda(image=color_noise_wrapper(amount, rgb), p=1)
tf_white_noise = A.RandomSnow(snow_point_lower=0,
                    snow_point_upper=0.3,
                    brightness_coeff=2.5,
                    always_apply=False,
                    p=1,
                )
tf_polygon_noise = A.Lambda(
                    image=polygon_noise_wrapper(
                        num_polygons, min_radius, max_radius
                    ),
                    p=1,
                )
tf_bright = A.RandomBrightnessContrast(
            brightness_limit=(factor, factor), contrast_limit=0, p=1
        )
tf_compose = A.Compose([tf_polygon_noise, tf_bright])

fnames = sorted(os.listdir(img_dir))
fname = fnames[img_idx]
img_path = os.path.join(img_dir, fname)

img = Image.open(img_path)
img = np.array(img)

fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
ax1.set_title("Original")
ax1.imshow(img)

aug_img = tf_compose(image=img)['image']

ax2 = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
ax2.set_title("Transformed")
ax2.imshow(aug_img)

if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

cv2.imwrite(
    os.path.join(save_dir, fname), cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
)